In [7]:
from utils.utils import preprocess_watch_data
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
from math import sin,cos
from numpy import random
import sys
from utils.utils import preprocess_watch_data
from utils.utils import get_yaw_pitch_roll
from utils.utils import get_rotation_matrix_from_yaw_pitch_roll
from utils.utils import get_rotated_basis
from utils.utils import get_tilt_correction_rotation_matrix_from_accelerometer
import numpy as np
import pandas as pd

df = preprocess_watch_data("drive.csv",save=False,plot=False)
df = df.fillna(method='ffill')

/tmp/ipykernel_38671/2424557251.py:16: DtypeWarning: Columns (41,49) have mixed types.Specify dtype option on import or set low_memory=False.
  df = preprocess_watch_data("drive.csv",save=False,plot=False)


In [8]:
acc = np.array(df[['acc_x','acc_y','acc_z']])
omega = np.array(df[['gyr_x','gyr_y','gyr_z']])
# df_omega = pd.DataFrame(omega)
# df_omega = df_omega.fillna(method="ffill")
# omega = np.array(df_omega)
time = df['gyr_t']
yaw_pitch_rolls = [np.zeros(3)]
Rs = [np.eye(3,3)]
R_tilts = [np.eye(3,3)]
bases = [np.eye(3,3)]
velocities = [np.zeros(3)]
positions = [np.zeros(3)]

for i,_ in enumerate(time):
    if (i==0):
        # don't have delta t, next
        continue
    delta_t = time[i]-time[i-1]
    yaw_pitch_roll = get_yaw_pitch_roll(i,omega,time,yaw_pitch_rolls)
    yaw_pitch_rolls.append(yaw_pitch_roll)
    R_tilt = get_tilt_correction_rotation_matrix_from_accelerometer(acc[i])
    R = get_rotation_matrix_from_yaw_pitch_roll(*yaw_pitch_roll)
    Rs.append(R)
    R_tilts.append(R_tilt)
    R = R @ R_tilt
    acc_global = R @ acc[i]
    acc_global = acc_global + np.array([0,0,1])
    velocity = acc_global*delta_t + velocities[i-1]
    velocities.append(velocity)
    position = velocity*delta_t + positions[i-1]
    positions.append(position)
    bases.append(get_rotated_basis(bases[0],R))

# from utils.utils import animate_trajectory
# animate_trajectory(time,bases,trajectory=positions)

# np.save('rpy.npy',np.array(yaw_pitch_rolls))
# np.save('pos.npy',np.array(positions))

In [6]:
p = np.array(positions).T
t = np.arange(len(p.T))
x = p[0]
y = p[1]
ax = plt.figure().add_subplot(projection='3d')

# df[df.isna().any(axis=1)]
p[0]

array([ 0.00000000e+00, -2.27344643e-07, -8.37905683e-07, ...,
        6.96315134e+08,  6.96315106e+08,  6.96315079e+08])